In [56]:
import sklearn.preprocessing
# import utils
import collections
import codecs
# import utils_nlp
import re
import time
import token
import os
import pickle
import random
import numpy as np
path_train = '../data/CoNLL2003/eng.train'
path_eval = '../data/CoNLL2003/eng.testa'
path_test = '../data/CoNLL2003/eng.testb'

### Here we analysis the vocabulary of CoNLL 2003 dataset for:
 * token
 * character
 * label<br><br>

And compare the token vocabulary OOV rate with glove.6B


In [57]:
# first 10 lines in test file
! head -20 ../data/CoNLL2003/eng.train

-DOCSTART- -X- O O

EU NNP I-NP I-ORG
rejects VBZ I-VP O
German JJ I-NP I-MISC
call NN I-NP O
to TO I-VP O
boycott VB I-VP O
British JJ I-NP I-MISC
lamb NN I-NP O
. . O O

Peter NNP I-NP I-PER
Blackburn NNP I-NP I-PER

BRUSSELS NNP I-NP I-LOC
1996-08-22 CD I-NP O

The DT I-NP O
European NNP I-NP I-ORG


In [58]:
def get_vocabs(filepath, sparator = ' ', lowercase = True, convert_digit = True):
    
    count_token = collections.Counter()
    count_label = collections.Counter()
    count_character = collections.Counter()
    
    if filepath:
        f = codecs.open(filepath, 'r', 'UTF-8')
        for line in f:
            line = line.strip().split(sparator)

            #skip sentence separator
            if len(line) == 0 or len(line[0]) == 0 or '-DOCSTART-' in line[0]:
                continue
            
            token = str(line[0])
            for character in token:
                count_character.update({character: 1})
                
#             lowercase & digit
            if lowercase:
                token = str(line[0]).lower()
            else:
                token = str(line[0])
            if convert_digit:
                if token.isdigit():
                    token = '$NUM$'
                
            label = str(line[-1])
            count_token.update({token: 1})
            count_label.update({label: 1})              
        
        f.close()    
            
    return count_token, count_label, count_character

In [59]:
count_token = {} 
count_label = {} 
count_character = {}

datasets = [('train',path_train), ('eval', path_eval), ('test', path_test)]
for dataset in datasets:
    count_token[dataset[0]], count_label[dataset[0]], count_character[dataset[0]] = get_vocabs(
        dataset[1], lowercase = False, convert_digit = False)
    
vocabs_corpus_token = count_token['train'] + count_token['test'] + count_token['eval']
vocabs_label = count_label['train'] + count_label['test'] + count_label['eval']
vocabs_char = count_character['train'] + count_character['test'] + count_character['eval']

### Label vocabulary

In [60]:
print('vocabs size of CoNLL dataset: ',len(vocabs_corpus_token))

vocabs size of CoNLL dataset:  30289


### Label vocabulary

In [61]:
print('Label size of CoNLL dataset: ',len(vocabs_label))
vocabs_label

Label size of CoNLL dataset:  8


Counter({'B-LOC': 17,
         'B-MISC': 50,
         'B-ORG': 29,
         'I-LOC': 12299,
         'I-MISC': 6729,
         'I-ORG': 14584,
         'I-PER': 17050,
         'O': 250660})

### Character vocabulary

In [62]:
print('character size of CoNLL dataset: ',len(vocabs_char))
vocabs_char

character size of CoNLL dataset:  85


Counter({'!': 4,
         '"': 3239,
         '#': 1,
         '$': 621,
         '%': 27,
         '&': 55,
         "'": 2949,
         '(': 4229,
         ')': 4229,
         '*': 67,
         '+': 235,
         ',': 12091,
         '-': 12245,
         '.': 17973,
         '/': 754,
         '0': 10865,
         '1': 13482,
         '2': 9085,
         '3': 6113,
         '4': 5298,
         '5': 5589,
         '6': 6898,
         '7': 4333,
         '8': 4600,
         '9': 7108,
         ':': 1449,
         ';': 137,
         '=': 49,
         '?': 20,
         '@': 5,
         'A': 9703,
         'B': 4925,
         'C': 6746,
         'D': 3963,
         'E': 6256,
         'F': 3174,
         'G': 3511,
         'H': 3174,
         'I': 6812,
         'J': 2048,
         'K': 2381,
         'L': 4690,
         'M': 5364,
         'N': 6281,
         'O': 5117,
         'P': 3980,
         'Q': 193,
         'R': 5986,
         'S': 10484,
         'T': 8157,
         'U': 2836

<div class="alert alert-success" align="justify">
the character vocabulary in CoNLL 2003 data set is small than UTF-8-128, so i choose to build our characters embedding table from the character in corpus, not from UTF-8-128, since the UTF-8-128 also contain some un-visible character that make no means in understand the text.
</div>

### Word vocabulary

In [63]:
def get_glove_vocab(filename):
    vocab = set()
    with open(filename) as f:
        for line in f:
            word = line.strip().split(' ')[0]
            vocab.add(word)
    return vocab

In [64]:
# glove.6B.100d vocab 
vocab_glove = get_glove_vocab('../data/glove/glove.6B.100d.txt')
total_token_glove = len(vocab_glove)
total_token_corpus = len(list(vocabs_corpus_token.keys()))

print('vocab size of glove:', total_token_glove)
print('vocab size of CoNLL:',total_token_corpus)

vocab size of glove: 400000
vocab size of CoNLL: 30289


In [65]:
OOV = set(vocabs_corpus_token.keys()) - vocab_glove
print('OOV rate in CoNLL compare with glove :{}%'.format((np.sum([vocabs_corpus_token[x] for x in OOV])/sum(vocabs_corpus_token.values()))*100))

OOV rate in CoNLL compare with glove :24.002216191468325%


In [66]:
len(OOV)

15671

In [67]:
# save OVV to a file
my_list = sorted(list(OOV))
with open('OOV_CoNLL_vs_glove.txt', 'w') as f:
    for item in my_list:
        f.write("%s\n" % item)

#### Digit rate in CoNLL

In [68]:
count_token = {} 
count_label = {} 
count_character = {}

datasets = [('train',path_train), ('eval', path_eval), ('test', path_test)]
for dataset in datasets:
    count_token[dataset[0]], count_label[dataset[0]], count_character[dataset[0]] = get_vocabs(
        dataset[1], lowercase = False, convert_digit = True)
    
vocabs_corpus_token1 = count_token['train'] + count_token['test'] + count_token['eval']
vocabs_label = count_label['train'] + count_label['test'] + count_label['eval']
vocabs_char = count_character['train'] + count_character['test'] + count_character['eval']

In [69]:
print('digit rate in CoNLL :{}%'.format((vocabs_corpus_token1['$NUM$']/sum(vocabs_corpus_token1.values()))*100))

digit rate in CoNLL :5.981726373342004%


in CoNLL dataset it has many different digit formats like:
* '9.5',
* '73.83',
* '4:13.353',
* '14,668,000',
* '312-408-8787',
* '32.0-43.0'.

<div class="alert alert-success" align="justify">
the digit consist most of the OOV token in CoNLL data set.
</div>

In [70]:
def find_all_digit(filepath, sparator = ' ', lowercase = True):
    
    digit = []
    if filepath:
        f = codecs.open(filepath, 'r', 'UTF-8')
        for line in f:
            line = line.strip().split(sparator)

            #skip sentence separator
            if len(line) == 0 or len(line[0]) == 0 or '-DOCSTART-' in line[0]:
                continue
                

            token = str(line[0])
            token1 = token
            if len(token) > 1 and any(char.isdigit() for char in token):
                token = token.replace('-','1')
                token = token.replace(',','1')
                token = token.replace('.','1')
                token = token.replace(':','1')
            if token.isdigit():
                digit.append(token1)
                
        f.close()    
            
    return digit

In [71]:
digits = {}
datasets = [('train',path_train), ('eval', path_eval), ('test', path_eval)]
for dataset in datasets:
    digits[dataset[0]] = find_all_digit(dataset[1])
    
vocabs_digits = digits['train'] + digits['test'] + digits['eval']
vocabs_digits

['1996-08-22',
 '47,600',
 '4,275',
 '10',
 '17,000',
 '1996-08-22',
 '17,000',
 '10,925',
 '16,935',
 '1966',
 '1967',
 '16',
 '1966',
 '1969',
 '5,060',
 '7,845',
 '1970',
 '27',
 '1996-08-22',
 '1996-08-22',
 '14.2',
 '1996-08-22',
 '14.2',
 '356,725',
 '1996',
 '304,850',
 '15,613',
 '13.6',
 '2.2',
 '1995',
 '32.7',
 '77,719',
 '49,269',
 '16.4',
 '35,563',
 '11.7',
 '1996',
 '3,420',
 '5522',
 '554',
 '643',
 '1996-08-22',
 '3311812-4',
 '100',
 '1996-08-22',
 '100',
 '6.625',
 '100.92',
 '1.875',
 '99.32',
 '1-10-100',
 '0.275',
 '1.60',
 '7.0',
 '2001',
 '171',
 '542',
 '7658',
 '300',
 '1999',
 '1996-08-22',
 '300',
 '12.5',
 '99.956',
 '10',
 '1-10-100',
 '2',
 '5',
 '5',
 '171',
 '542',
 '8863',
 '10',
 '24',
 '1996-08-22',
 '1967',
 '1991',
 '1996-08-22',
 '100',
 '800',
 '1996-08-22',
 '1991',
 '100',
 '1996-08-22',
 '3.7504',
 '06',
 '1996-08-22',
 '1996-08-22',
 '1996-08-22',
 '1995',
 '76',
 '1996-08-22',
 '1990',
 '1995',
 '22',
 '1996-08-22',
 '12',
 '22',
 '1996-08-2

In [72]:
print('General digit rate in CoNLL  :{}%'.format((len(vocabs_digits)/sum(vocabs_corpus_token1.values()))*100))

General digit rate in CoNLL  :8.556224246727135%


#### and digit should not count some strings like
* 84th
* 20-month
* 25-YEAR